<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher_New/blob/main/optPhotoFiles_cleaning20231025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#optPhotoFilesの調整ver.2
前回の調整で多くの画像ファイルがoptPhotoTable_controlsとの整合性を失い、多くの属性データを失った。その理由は最初に不良ファイルのパージを行い、ファイル名称の統一を後回しにしたことが原因だった。このため、今回は最初にバックアップから元データを起こしたのち、すぐにその名称を統一し、ファイル操作とcsvファイル操作が連動させやすい状況にする。

#Google_Driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#ライブラリのダウンロードおよびインポート

In [2]:
!pip install pyheif
!pip install rembg
import os
import shutil
from PIL import Image
import pyheif
import pandas as pd
import csv
from rembg import remove
from concurrent.futures import ProcessPoolExecutor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00


#バックアップデータの呼び出し
バックアップとして保存されているoptPhotoFilesBackUpに格納されているnonRA_Right_HandとnonRA_Left_Handを呼び出し、空のディレクトリ"optPhotoFiles"に保存する。

In [11]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/optPhotoFiles/nonRA_Right_Hand"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand"

if not os.path.exists(dst_dir):
    shutil.copytree(src_dir, dst_dir)

In [6]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/optPhotoFiles/nonRA_Right_Hand"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/optPhotoFiles/nonRA_Right_Hand: 1208


In [4]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand: 1208


In [7]:
import os
import pandas as pd

# 指定したディレクトリのパス
directory_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/optPhotoFiles/nonRA_Right_Hand'

# ディレクトリ内のファイル名をリストに格納
file_names = os.listdir(directory_path)

# リストをDataFrameに変換
df = pd.DataFrame(file_names, columns=['File Names'])
# 保存先のパスを指定
save_path = '/content/drive/MyDrive/file_names.xlsx'

# DataFrameをExcelファイルとして保存
df.to_excel(save_path, index=False)

同じ作業を左手にも行う。

In [ ]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/optPhotoFiles/nonRA_Left_Hand"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand"

if not os.path.exists(dst_dir):
    shutil.copytree(src_dir, dst_dir)

In [4]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand: 1200


#画像ファイルの名称統一
nonRA_Right_HandとnonRA_Left_Handには、さまざまなファイル形式の画像ファイルが格納されており、その名称もまちまちで規則性に欠ける。まず、それらの画像ファイルとdemographic_dataを結びつけるために、ファイル名称を"nonRA_right_hand_番号_.拡張子"の形式に統一する。このファイル名は"right_hand_filename"として既存のデータテーブル"optPhotoTable_controls.csv"に保存する。

作業用の複製を作成する。

In [6]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_1st"

if not os.path.exists(dst_dir):
    shutil.copytree(src_dir, dst_dir)

複製時にファイルの脱落がないことを確認するため、複製後のファイル数を確認する。

In [7]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_1st"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_1st: 1208


同じ作業を左手にも行う。

In [14]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_1st"

if not os.path.exists(dst_dir):
    shutil.copytree(src_dir, dst_dir)

複製後のファイル数を確認する。

In [15]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_1st"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_1st: 1200


さらに、使用するcsvファイルも、破損がないものを'optPhotoTable_back_up'と命名したディレクトリに保存し、ここから作業の都度、MyDriveの外にコピーしたoptPhotoTable内のcsvを使用することとする。

In [8]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoTable/optPhotoTable_back_up"
dst_dir = "/content/optPhotoTable"

if not os.path.exists(dst_dir):
    shutil.copytree(src_dir, dst_dir)

複製後のファイル数を確認する。

In [9]:
# 対象のディレクトリ
target_dir = "/content/optPhotoTable"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/optPhotoTable: 8


作業を開始する。

In [8]:
import pandas as pd
import os
import shutil

# CSVデータを読み込む
csv_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoTable/optPhotoTable_for_use/optPhotoTable_controls_1203cases_.csv'
data = pd.read_csv(csv_path)

# right_hand_filenameからファイル名（拡張子を除く）を取得
data['filename_without_ext'] = data['right_hand_filename'].str.rsplit(pat='.', n=1).str[0]

# 画像ファイルの一覧を取得
image_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_1st'
image_files = os.listdir(image_dir)
image_files_without_ext = [os.path.splitext(file)[0] for file in image_files]

# 出力ディレクトリを確認し、存在しなければ作成
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2nd'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 新しいファイル名を格納する辞書
new_filename_dict = {}

# ファイル名の照合と変更
for index, image_name_without_ext in enumerate(image_files_without_ext):
    if image_name_without_ext in data['filename_without_ext'].values:
        ext = os.path.splitext(image_files[index])[1]
        new_filename = f"nonRA_right_hand_{index + 1}{ext}"
        new_filename_dict[image_name_without_ext] = new_filename

        # ファイルをコピー
        old_filepath = os.path.join(image_dir, image_files[index])
        new_filepath = os.path.join(output_dir, new_filename)
        shutil.copy2(old_filepath, new_filepath)

# 新しいファイル名をCSVデータに追加
data['new_right_hand_filename'] = data['filename_without_ext'].map(new_filename_dict)

# CSVファイルを保存
data.to_csv(csv_path, index=False)

作業時にファイルの脱落がないことを確認するため、複製後のファイル数を確認する。

In [9]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2nd"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2nd: 1090


In [10]:
import pandas as pd
import os

# CSVデータを読み込む
csv_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoTable/optPhotoTable_for_use/optPhotoTable_controls_1203cases_.csv'
data = pd.read_csv(csv_path)

# right_hand_filenameのファイル名（拡張子を除く）をリスト化
csv_files_without_ext = data['right_hand_filename'].str.rsplit(pat='.', n=1).str[0].tolist()

# nonRA_Right_Hand_1stのファイル一覧を取得
dir_1st = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_1st'
files_1st = os.listdir(dir_1st)
files_1st_without_ext = [os.path.splitext(file)[0] for file in files_1st]

# 照合されなかったファイル名をリストアップ
unmatched_files = [file for file in files_1st_without_ext if file not in csv_files_without_ext]

# 結果を表示
print(f"Number of unmatched files: {len(unmatched_files)}")
print("\n".join(sorted(unmatched_files)))


Number of unmatched files: 118
.DS_Store
139092 - Wくろーばー
16935491218270 - ダダダ
16936887621261567231452185773132 - Daisuke Sasaki (ダイスケ)
16938056130092956529229493354723 - マドレーヌ
16938263423815238061394904543704 - もふもふうさぎ
16938937785752280696060118810028 - ローストビーフ
16939233555418178658477424367657 - 豊坂竹寿（プン）
16939700237033007148771765966352 - あきぶー
16941753974997621506871000529839 - ランキング動画スタトレch
16942294062097119172031907879149 - テキベンジャー
1694417994680 - 夏バテ子
20230901-1 - えび
20230901_142527 - パテマ
20230902_225705 - 烏宮わかば
20230905_104007 - いずみ
20230905_223809 - だいちゃん
20230906_113753 - ぱるもも
20230909_162120_ - リアル知人にバレてはいけない趣味アカ
230903101922155 - かず
230908065307680 - たなぼた
31CC6BE9-F2D0-4F49-B8EB-0247E2697F48 - ろびん
32369503-D053-4350-AA3F-0C82BC8E5C1F - さけるちーず
42565 - パシフィックチューブ
5057592E-C0DF-41C1-B660-AE7C0F943F2A - ポテトへっど
579EBF9E-071A-47A9-B3B0-330ABF8C3583 - でこぼーこ
B50A0BD4-EA63-42F1-9F4C-A0DFCAA65D71 - やまだかず
DSC01403 - ごまむぎ
DSC_0004 - クラウドワークス仕事
DSC_0048~2

In [11]:
import pandas as pd
import os
import shutil

# 照合されなかったファイルのリストを取得
csv_path =  '/content/drive/MyDrive/OptPhotoFiles/optPhotoTable/optPhotoTable_for_use/optPhotoTable_controls_1203cases_.csv'
data = pd.read_csv(csv_path)
csv_files_without_ext = data['right_hand_filename'].str.rsplit(pat='.', n=1).str[0].tolist()

dir_1st = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_1st'
files_1st = os.listdir(dir_1st)
files_1st_without_ext = [os.path.splitext(file)[0] for file in files_1st]

unmatched_files = [file for file in files_1st if os.path.splitext(file)[0] not in csv_files_without_ext]

# ファイルの改名と移動
new_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2000th'
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

rename_map = {}
counter = 2001

for file in unmatched_files:
    old_path = os.path.join(dir_1st, file)
    file_ext = os.path.splitext(file)[1]
    new_filename = f"nonRA_right_hand_{counter}{file_ext}"
    new_path = os.path.join(new_dir, new_filename)
    shutil.move(old_path, new_path)

    rename_map[file] = new_filename
    counter += 1

df_rename_map = pd.DataFrame(list(rename_map.items()), columns=['Original', 'Renamed'])

# ファイル名のマッピングをCSVに保存
csv_save_path = '/content/rename_mapping_right_hand.csv'
df_rename_map.to_csv(csv_save_path, index=False)

print(f"Renaming and moving completed. Mapping saved to {csv_save_path}")

# マッピングの辞書をDataFrameに変換
df_rename_map = pd.DataFrame(list(rename_map.items()), columns=['Original', 'Renamed'])

Renaming and moving completed. Mapping saved to /content/rename_mapping_right_hand.csv


nonRA_Left_Hand_2000に含まれるファイル数を確認する。

In [23]:
# 対象のディレクトリ
target_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2000th'

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2000th: 132


左手の画像ファイルにも同様の作業を行う。

In [24]:
import pandas as pd
import os
import shutil

# CSVデータを読み込む
csv_path = '/content/optPhotoTable/optPhotoTable_controls.csv'
data = pd.read_csv(csv_path)

# right_hand_filenameからファイル名（拡張子を除く）を取得
data['filename_without_ext'] = data['left_hand_filename'].str.rsplit(pat='.', n=1).str[0]

# 画像ファイルの一覧を取得
image_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_1st'
image_files = os.listdir(image_dir)
image_files_without_ext = [os.path.splitext(file)[0] for file in image_files]

# 出力ディレクトリを確認し、存在しなければ作成
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2nd'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 新しいファイル名を格納する辞書
new_filename_dict = {}

# ファイル名の照合と変更
for index, image_name_without_ext in enumerate(image_files_without_ext):
    if image_name_without_ext in data['filename_without_ext'].values:
        ext = os.path.splitext(image_files[index])[1]
        new_filename = f"nonRA_left_hand_{index + 1}{ext}"
        new_filename_dict[image_name_without_ext] = new_filename

        # ファイルをコピー
        old_filepath = os.path.join(image_dir, image_files[index])
        new_filepath = os.path.join(output_dir, new_filename)
        shutil.copy2(old_filepath, new_filepath)

# 新しいファイル名をCSVデータに追加
data['new_left_hand_filename'] = data['filename_without_ext'].map(new_filename_dict)

# CSVファイルを保存
data.to_csv(csv_path, index=False)

作業時にファイルの脱落がないことを確認するため、複製後のファイル数を確認する。

In [25]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2nd"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2nd: 1071


In [26]:
import pandas as pd
import os

# CSVデータを読み込む
csv_path = '/content/optPhotoTable/optPhotoTable_controls.csv'
data = pd.read_csv(csv_path)

# right_hand_filenameのファイル名（拡張子を除く）をリスト化
csv_files_without_ext = data['left_hand_filename'].str.rsplit(pat='.', n=1).str[0].tolist()

# nonRA_Left_Hand_1stのファイル一覧を取得
dir_1st = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_1st'
files_1st = os.listdir(dir_1st)
files_1st_without_ext = [os.path.splitext(file)[0] for file in files_1st]

# 照合されなかったファイル名をリストアップ
unmatched_files = [file for file in files_1st_without_ext if file not in csv_files_without_ext]

# 結果を表示
print(f"Number of unmatched files: {len(unmatched_files)}")
print("\n".join(sorted(unmatched_files)))

Number of unmatched files: 129
.DS_Store
11D725DD-470E-4D54-A9C6-86CC7678E5BE - ちゃんぽん
139091 - Wくろーばー
16935491220241 - ダダダ
16936888001023448702726803656649 - Daisuke Sasaki (ダイスケ)
16938056831914806607150442218909 - マドレーヌ
16938263744198438720843662757159 - もふもふうさぎ
16938938537626922193867164875586 - ローストビーフ
16939233990632103362032329564222 - 豊坂竹寿（プン）
16939700779793150902473308906639 - あきぶー
16941754264587574177806647020502 - ランキング動画スタトレch
16942294671437814175898042002335 - テキベンジャー
1694417996805 - 夏バテ子
20230901-2 - えび
20230901_142513 - パテマ
20230902_225851 - 烏宮わかば
20230905_104029 - いずみ
20230905_223829 - だいちゃん
20230906_113651 - ぱるもも
20230909_162135_ - リアル知人にバレてはいけない趣味アカ
230903101853844 - かず
230908065002370 - たなぼた
42566 - パシフィックチューブ
50145A14-CD15-4640-AF2F-62291DCEC0BB - でこぼーこ
6489F3E0-23AB-43D0-986B-EEF6B7FC6B23 - ろびん
726BABDA-AE8E-4B9A-AC08-69C80EDAFB27 - ポテトへっど
9A971FD8-658A-4E66-83E8-6935A6CB2FBD - やまだかず
DSC01400 - ごまむぎ
DSC_0000_BURST20230903161433497_CO

In [27]:
import pandas as pd
import os
import shutil
##失われたデータを同定する。
#指定されたパスのCSVファイルを指すための変数"csv_path"を設定。
csv_path =  '/content/optPhotoTable/optPhotoTable_controls.csv'
#Pandasライブラリを使用して、直前に"csv_path"で指定されたパスのCSVファイルを読み込み、"data"というDataFrameに格納。
data = pd.read_csv(csv_path)
#"left_hand_filename"カラムのファイル名から拡張子を除去し、リスト化する。
csv_files_without_ext = data['left_hand_filename'].str.rsplit(pat='.', n=1).str[0].tolist()
#操作対象のディレクトリのパスを指すための変数、"dir_1st"を設定。
dir_1st = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_1st'
#上で指定したディレクトリ内のファイルのリスト、"files_1st"を取得。
files_1st = os.listdir(dir_1st)
#ディレクトリ内の各ファイル名から拡張子を除去。
files_1st_without_ext = [os.path.splitext(file)[0] for file in files_1st]
#ディレクトリ内のファイルに含まれているにもかかわらず、CSVの"left_hand_filename"カラムのファイル名に記載されていないファイルのリスト、"unmatched_files"を取得。
unmatched_files = [file for file in files_1st if os.path.splitext(file)[0] not in csv_files_without_ext]

# ファイルの改名と移動
#新しくファイルを移動させるディレクトリのパスを指すための変数を設定。
new_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2000th'
#新しいディレクトリが存在しない場合、ディレクトリを作成。
if not os.path.exists(new_dir):
    os.makedirs(new_dir)
#改名前後のファイル名をマッピングするための辞書を初期化。
rename_map = {}
#新しいファイル名を生成する際のカウンターを初期化。
counter = 2001

#CSVから読み出せなかったファイル、"unmatched_files"についてのループを開始。
for file in unmatched_files:
    #現在のファイルのパスを取得。
    old_path = os.path.join(dir_1st, file)
    #ファイルの拡張子を取得。
    file_ext = os.path.splitext(file)[1]
    #新しいファイル名を生成。
    new_filename = f"nonRA_left_hand_{counter}{file_ext}"
    #新しいファイルのパスを取得。
    new_path = os.path.join(new_dir, new_filename)
    #ファイルを新しい場所に移動。
    shutil.move(old_path, new_path)
    #改名前後のファイル名を辞書に追加。
    rename_map[file] = new_filename
    #カウンターをインクリメント。
    counter += 1

# マッピングの辞書をDataFrameに変換
df_rename_map = pd.DataFrame(list(rename_map.items()), columns=['Original', 'Renamed'])

# ファイル名のマッピングをCSVに保存
csv_save_path = '/content/rename_mapping_left_hand.csv'
df_rename_map.to_csv(csv_save_path, index=False)

print(f"Renaming and moving completed. Mapping saved to {csv_save_path}")

Renaming and moving completed. Mapping saved to /content/rename_mapping_left_hand.csv


nonRA_Left_Hand_2000に含まれるファイル数を確認する。

In [28]:
# 対象のディレクトリ
target_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2000th'

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2000th: 129


次の作業に移る前に、ここまでで出来上がったファイル をコピーして、バックアップとする。対象となるファイル群は以下の通り。
'rename_mapping_left_hand.csv'

'rename_mapping_right_hand.csv'

'optPhotoTable_controls.csv'

'nonRA_Left_Hand_2000th'

'nonRA_Right_Hand_2000th'

nonRA_Left_Hand_2nd

nonRA_Right_Hand_2nd

これを施行することで、作業は次の段階に進行可能となる。

In [12]:
shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2000th', '/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/nonRA/nonRA_Right_Hand_2000th')

#shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2000th', '/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/nonRA/nonRA_Left_Hand_2000th')

shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2nd', '/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/nonRA/nonRA_Right_Hand_2nd')

#shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2nd', '/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/nonRA/nonRA_Left_Hand_2nd')

'/content/drive/MyDrive/OptPhotoFiles/optPhotoFilesBackUp/nonRA/nonRA_Right_Hand_2nd'


#重要！！！！ここにTable改変の手作業を要する。
クリーニング作業の終了したファイル名に書き換えた新しいcsvファイルをローカル環境で作成し、optPhotoTable_back_upにアップロードする。

#"nonRA_Right_Hand_2nd"と、"nonRA_Right_Hand_2000th"を結合
バックアップファイルを作成したのち、"nonRA_Right_Hand_2nd"と、"nonRA_Right_Hand_2000th"を結合し、"nonRA_Right_Hand_3rd"を作成する。以降はこのファイルを使用して作業を進める。

In [13]:
import os
import shutil

# ディレクトリのパスを定義
src_dir1 = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2000th'
src_dir2 = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_2nd'
dst_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_3rd'

# nonRA_Right_Hand_3rdディレクトリを作成（既に存在していた場合は作成しない）
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# nonRA_Right_Hand_2000thとnonRA_Right_Hand_2ndからファイルを取得し、nonRA_Right_Hand_3rdにコピー
for src in [src_dir1, src_dir2]:
    for filename in os.listdir(src):
        file_path = os.path.join(src, filename)
        if os.path.isfile(file_path):
            shutil.copy(file_path, dst_dir)

print("コピーが完了しました！")

コピーが完了しました！


In [14]:
# 対象のディレクトリ
target_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_3rd'
# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_3rd: 1208


In [37]:
# ディレクトリのパスを定義
src_dir1 = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2000th'
src_dir2 = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_2nd'
dst_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_3rd'

# nonRA_Right_Hand_3rdディレクトリを作成（既に存在していた場合は作成しない）
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# nonRA_Right_Hand_2000thとnonRA_Right_Hand_2ndからファイルを取得し、nonRA_Right_Hand_3rdにコピー
for src in [src_dir1, src_dir2]:
    for filename in os.listdir(src):
        file_path = os.path.join(src, filename)
        if os.path.isfile(file_path):
            shutil.copy(file_path, dst_dir)

print("コピーが完了しました！")

コピーが完了しました！


In [38]:
# 対象のディレクトリ
target_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_3rd'
# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_3rd: 1200


#pngファイルを探し出し、jpgに変換する。
optPhotoFilesには、さまざまなファイル形式の画像が格納されている。jpgが最多だが、いくつかのpngが混在している。これを探し出し、jpgに変換する。

まず、右手のファイルに作業を行う。

In [15]:
import os
from PIL import Image

# ディレクトリのパスを定義
dir_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_3rd'

# .pngファイルを探し出す
png_files = [f for f in os.listdir(dir_path) if f.endswith('.png')]

# .pngファイルを.jpgに変換
for png_file in png_files:
    png_path = os.path.join(dir_path, png_file)
    # 画像を読み込む
    img = Image.open(png_path)
    # .jpgとして保存
    jpg_path = os.path.join(dir_path, os.path.splitext(png_file)[0] + '.jpg')
    img.convert('RGB').save(jpg_path, 'JPEG')
    # .pngファイルを削除
    os.remove(png_path)

print(f"{len(png_files)}個の.pngファイルを.jpgに変換しました！")

8個の.pngファイルを.jpgに変換しました！


左手でも同じ作業を行う。

In [8]:
# ディレクトリのパスを定義
dir_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_3rd'

# .pngファイルを探し出す
png_files = [f for f in os.listdir(dir_path) if f.endswith('.png')]

# .pngファイルを.jpgに変換
for png_file in png_files:
    png_path = os.path.join(dir_path, png_file)
    # 画像を読み込む
    img = Image.open(png_path)
    # .jpgとして保存
    jpg_path = os.path.join(dir_path, os.path.splitext(png_file)[0] + '.jpg')
    img.convert('RGB').save(jpg_path, 'JPEG')
    # .pngファイルを削除
    os.remove(png_path)

print(f"{len(png_files)}個の.pngファイルを.jpgに変換しました！")

7個の.pngファイルを.jpgに変換しました！


#pyheifを用いたheic形式ファイルをjpg形式に変換する。

#ライブラリのインポート
pyheifをはじめとする必要なライブラリをインポートする。

In [16]:
!pip install pyheif

import os
import pyheif
from PIL import Image

#右手ファイルの作業
右手の作業から開始する。

In [17]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_3rd"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_4th"

# ディレクトリが存在しない場合、ディレクトリを作成
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

failed_files = []

for root, _, files in os.walk(src_dir):
    for file in files:
        src_path = os.path.join(root, file)
        filename, ext = os.path.splitext(file)

        try:
            # Convert heic files to jpg
            if ext.lower() == '.heic':
                heif_file = pyheif.read(src_path)
                image = Image.frombytes(
                    heif_file.mode,
                    heif_file.size,
                    heif_file.data,
                    "raw",
                    heif_file.mode,
                    heif_file.stride,
                )
                dst_path = os.path.join(dst_dir, filename + '.jpg')
                image.save(dst_path, "JPEG")

            # Rename other jpg files to ensure the extension is ".jpg"
            elif ext.lower() in ['.jpeg', '.jpg']:
                dst_path = os.path.join(dst_dir, filename + '.jpg')
                image = Image.open(src_path)
                image.save(dst_path, "JPEG")

        except Exception as e:
            print(f"Error processing file {file}: {e}")
            failed_files.append(file)

if failed_files:
    print("Failed to process the following files:")
    for file in failed_files:
        print(file)
else:
    print("All files processed successfully!")

Error processing file nonRA_right_hand_128.HEIC: Input is not a HEIF/AVIF file
Failed to process the following files:
nonRA_right_hand_128.HEIC


nonRA_Right_Hand_4thに含まれるファイル数を確認する。

In [18]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_4th"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_4th: 1206


#左手ファイルの作業
左手にも同様の作業を施行する。

In [10]:
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_3rd"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_4th"

# ディレクトリが存在しない場合、ディレクトリを作成
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

failed_files = []

for root, _, files in os.walk(src_dir):
    for file in files:
        src_path = os.path.join(root, file)
        filename, ext = os.path.splitext(file)

        try:
            # Convert heic files to jpg
            if ext.lower() == '.heic':
                heif_file = pyheif.read(src_path)
                image = Image.frombytes(
                    heif_file.mode,
                    heif_file.size,
                    heif_file.data,
                    "raw",
                    heif_file.mode,
                    heif_file.stride,
                )
                dst_path = os.path.join(dst_dir, filename + '.jpg')
                image.save(dst_path, "JPEG")

            # Rename other jpg files to ensure the extension is ".jpg"
            elif ext.lower() in ['.jpeg', '.jpg']:
                dst_path = os.path.join(dst_dir, filename + '.jpg')
                image = Image.open(src_path)
                image.save(dst_path, "JPEG")

        except Exception as e:
            print(f"Error processing file {file}: {e}")
            failed_files.append(file)

if failed_files:
    print("Failed to process the following files:")
    for file in failed_files:
        print(file)
else:
    print("All files processed successfully!")

Error processing file nonRA_left_hand_136.HEIC: Input is not a HEIF/AVIF file
Failed to process the following files:
nonRA_left_hand_136.HEIC


nonRA_Left_Hand_4thに含まれるファイル数を確認する。

In [11]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_4th"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_4th: 1198


#rembgライブラリを使用した背景の削除
/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Handに含まれている画像は、非常に雑多な背景のなかに手背の画像が撮影されている。背景が学習に悪影響を及ぼさないように、背景を削除する。

まず、右手画像の背景除去から開始する。

In [19]:
!pip install rembg

import os
from rembg import remove
from concurrent.futures import ProcessPoolExecutor

def remove_bg_from_file(file_path, dst_dir):
    with open(file_path, "rb") as f:
        img = f.read()
        output = remove(img)

    dst_file_path = os.path.join(dst_dir, os.path.basename(file_path))
    with open(dst_file_path, "wb") as f_out:
        f_out.write(output)

# 画像の背景を削除するディレクトリ
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_4th"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_5th"

# ディレクトリが存在しない場合、新規作成
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 14.4GB/s]



Background removal and saving to new directory completed!


nonRA_Right_Hand_5thに含まれるファイル数を確認する。

In [20]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_5th"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_5th: 1206


次に左手画像の背景除去を行う。

In [14]:
# 画像の背景を削除するディレクトリ
src_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_4th"
dst_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_5th"

# ディレクトリが存在しない場合、新規作成
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

Background removal and saving to new directory completed!


nonRA_Right_Hand_4thに含まれるファイル数を確認する。

In [15]:
# 対象のディレクトリ
target_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_5th"

# ファイル数を計算
file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

print(f"Total number of files in {target_dir}: {file_count}")

Total number of files in /content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand_5th: 1198


#CSVファイルの改変

In [25]:
import pandas as pd

# CSVファイルのパス
csv_path = "/content/drive/MyDrive/OptPhotoFiles/optPhotoTable/optPhotoTable_for_use/optPhotoTable_controls_1203cases_final.csv"

# CSVファイルを読み込む
df = pd.read_csv(csv_path)

# 4列目と5列目の拡張子を.jpgに変更
df['right_hand_filename'] = df['right_hand_filename'].str.replace(r'\.\w+$', '.jpg')
#df['left_hand_filename'] = df['left_hand_filename'].str.replace(r'\.\w+$', '.jpg')

# CSVファイルを上書き保存
df.to_csv(csv_path, index=False)

print("拡張子の変更が完了しました！")

拡張子の変更が完了しました！


<ipython-input-25-a9aa8b6cc9d4>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['right_hand_filename'] = df['right_hand_filename'].str.replace(r'\.\w+$', '.jpg')


In [23]:
import os
import pandas as pd

# 指定したディレクトリのパス
directory_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand_5th'

# ディレクトリ内のファイル名をリストに格納
file_names = os.listdir(directory_path)

# リストをDataFrameに変換
df = pd.DataFrame(file_names, columns=['File Names'])
# 保存先のパスを指定
save_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoTable/list_nonRA_Right_Hand_5th.xlsx'

# DataFrameをExcelファイルとして保存
df.to_excel(save_path, index=False)